# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,12.81,58,40,4.12,AR,1736029753
1,1,grytviken,-54.2811,-36.5092,4.23,67,15,5.79,GS,1736029754
2,2,talodi,10.6325,30.3797,26.06,22,27,2.55,SD,1736029755
3,3,port-aux-francais,-49.3500,70.2167,5.80,82,73,11.92,TF,1736029757
4,4,novyy urgal,51.0736,132.5642,-36.78,99,100,0.70,RU,1736029758


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:
# Install the required packages
%pip install cartopy geoviews pyproj

import geoviews as gv
import cartopy.crs as ccrs

Note: you may need to restart the kernel to use updated packages.


In [48]:

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    tiles='OSM', 
    size='Humidity', 
    color='Humidity', 
    hover_cols=['City', 'Country', 'Humidity']
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [49]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] >= 20) & 
    (city_data_df['Max Temp'] <= 30) & 
    (city_data_df['Wind Speed'] < 10) & 
    (city_data_df['Cloudiness'] < 50)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,talodi,10.6325,30.3797,26.06,22,27,2.55,SD,1736029755
10,10,imbituba,-28.2400,-48.6703,20.78,76,6,2.06,BR,1736029766
11,11,bikenibeu village,1.3673,173.1241,28.98,74,20,3.09,KI,1736029769
13,13,kombissiri,12.0656,-1.3375,22.93,21,10,2.95,BF,1736029772
14,14,kingston,17.9970,-76.7936,28.53,65,20,2.57,JM,1736029775


### Step 3: Create a new DataFrame called `hotel_df`.

In [50]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
2,talodi,SD,10.6325,30.3797,22,
10,imbituba,BR,-28.2400,-48.6703,76,
11,bikenibeu village,KI,1.3673,173.1241,74,
13,kombissiri,BF,12.0656,-1.3375,21,
14,kingston,JM,17.9970,-76.7936,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
import requests

In [56]:

geoapify_key = "03dda413cfb24be69c36ec0487789038"
base_url = "https://api.geoapify.com/v2/places"

# Set parameters to search for a hotel
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 5
}

# Define radius
radius = 20000

# Print a message to follow up the hotel search
hotel_df['Hotel Name'] = None
print("Searching for hotels...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    if response.status_code == 401:
        print(f"Error: Unauthorized request. Please verify your API key for {row['City']}.")
        hotel_df.at[index, 'Hotel Name'] = "Invalid API Key"

    elif response.status_code == 200:
        data = response.json()
        
        # Update the hotel name in the DataFrame
        if 'features' in data and len(data['features']) > 0:
            hotel_df.at[index, 'Hotel Name'] = data['features'][0]['properties'].get('name', 'No hotel name provided')
        else:
            hotel_df.at[index, 'Hotel Name'] = "No hotel found"
    
    else:
        print(f"Error {response.status_code} for {row['City']}")
        hotel_df.at[index, 'Hotel Name'] = "Error in API call"
    
    # Log the search results
    print(f"Hotel search for {row['City']} complete.")

# Display sample data
print("\nFinal Hotel Search Results:")
print(hotel_df)
hotel_df.head()


Searching for hotels...
Hotel search for talodi complete.
Hotel search for imbituba complete.
Hotel search for bikenibeu village complete.
Hotel search for kombissiri complete.
Hotel search for kingston complete.
Hotel search for adamstown complete.
Hotel search for umm kaddadah complete.

Final Hotel Search Results:
                 City Country      Lat       Lng  Humidity  \
2              talodi      SD  10.6325   30.3797        22   
10           imbituba      BR -28.2400  -48.6703        76   
11  bikenibeu village      KI   1.3673  173.1241        74   
13         kombissiri      BF  12.0656   -1.3375        21   
14           kingston      JM  17.9970  -76.7936        65   
18          adamstown      PN -25.0660 -130.1015        70   
21       umm kaddadah      SD  13.6017   26.6876        15   

                Hotel Name  
2           No hotel found  
10   Silvestre Praia Hotel  
11          Otintaai Hotel  
13  No hotel name provided  
14         Aphrodite Hotel  
18        

,City,Country,Lat,Lng,Humidity,Hotel Name
2,talodi,SD,10.6325,30.3797,22,No hotel found
10,imbituba,BR,-28.2400,-48.6703,76,Silvestre Praia Hotel
11,bikenibeu village,KI,1.3673,173.1241,74,Otintaai Hotel
13,kombissiri,BF,12.0656,-1.3375,21,No hotel name provided
14,kingston,JM,17.9970,-76.7936,65,Aphrodite Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [57]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    tiles='OSM', 
    size='Humidity', 
    color='Humidity', 
    hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)